In [1]:
import xarray as xr
import numpy as np 
from matplotlib import pyplot as plt
from matplotlib import cm

In [2]:
## Okay so I decided to make a new notebook because it's way easier to...
## ...organize things this way 

## Alright anywho, let's now look at the (somewhat) opposite proccess of primary production, which is respiration!
## We're going to use AOU (apparent oxygen utilization) as an indicator of respiration because..
    # A) Using the amount of accumulated carbon dioxide wouldn't be totally accurate since carbon dioxide levels gets too involved into carbonate chemistry 
    # Oxygen, on the other hand, is a reactant in the respiration equation/mechanism
    # AOU essentially measures the amount of oxygen that was used up in this respiration equation (more or less) and would be a sort of proxy to indicate how much..
    #..organic matter is being oxidized 
    # The AOU variable is an equation where the amount of oxygen that is respired is subtracted from the amount of oxygen in the ocean that's involved with sea-air exchange


In [3]:
### Alright, now that that's established, let's call in our directories:

rootdir = '/archive/oar.gfdl.cmip6/ESM4/DECK/ESM4_piControl_D/gfdl.ncrc4-intel16-prod-openmp/pp/'

# We're going to start with ANNUAL respiration

datadir_ann = 'ocean_cobalt_omip_tracers_year_z/ts/annual/5yr/'
filename_ann_o2 = 'ocean_cobalt_omip_tracers_year_z.0711-0715.o2.nc'
filename_ann_o2sat = 'ocean_cobalt_omip_tracers_year_z.0711-0715.o2sat.nc'
filename_ann_vol = 'ocean_cobalt_omip_tracers_year_z.0711-0715.volcello.nc'

datadir_temp = 'ocean_annual_z/ts/annual/5yr/'
filename_temp = 'ocean_annual_z.0711-0715.thetao.nc'

## MONTHLY respiration 

datadir_mon_aou = 'ocean_cobalt_omip_tracers_month_z/av/monthly_5yr/'
filename_mon_aou = 'ocean_cobalt_omip_tracers_month_z.0711-0715.*.nc'


datadir_mon_temp = 'ocean_monthly_z/av/monthly_5yr/'
filename_mon_temp = 'ocean_monthly_z.0711-0715.*.nc'


In [5]:
# Annual data 

aou_ann_o2 = xr.open_dataset(rootdir+datadir_ann+filename_ann_o2)
aou_ann_o2sat = xr.open_dataset(rootdir+datadir_ann+filename_ann_o2sat)
aou_ann_vol = xr.open_dataset(rootdir+datadir_ann+filename_ann_vol)

aou_ann_temp = xr.open_dataset(rootdir+datadir_temp+filename_temp)

In [6]:
# Monthly data

aou_mon_variables = xr.open_mfdataset(rootdir+datadir_mon_aou+filename_mon_aou)
aou_mon_temp = xr.open_mfdataset(rootdir+datadir_mon_temp+filename_mon_temp)

In [7]:
## Now we're going to create the datasets

# Annual data

dm = xr.merge([aou_ann_o2, aou_ann_o2sat, aou_ann_vol, aou_ann_temp])

# Monthly data

do = xr.merge([aou_mon_variables.o2, aou_mon_variables.o2sat, aou_mon_variables.volcello, aou_mon_temp.thetao])

/nbhome/Alexandra.Matthews/miniconda/envs/basic/lib/python3.7/site-packages/numpy/core/numeric.py:538: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/nbhome/Alexandra.Matthews/miniconda/envs/basic/lib/python3.7/site-packages/numpy/core/numeric.py:538: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/nbhome/Alexandra.Matthews/miniconda/envs/basic/lib/python3.7/site-packages/numpy/core/numeric.py:538: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/nbhome/Alexandra.Matthews/miniconda/envs/basic/lib/pyt

In [9]:
### Annual respiration x volume variables 
## This is going to be a little different from previous notebooks..
## ..where I only multiplied one variable by volume
## We're going to multiply both o2 and o2sat by volume to get the mole content
## We're also going to subtract o2 from o2sat


# Annual amount of moles 

o2V_ann = dm.o2 * dm.volcello
o2V_ann.name = 'o2V_ann'

o2satV_ann = dm.o2sat * dm.volcello
o2satV_ann.name = 'o2satV_ann'

AOU_ann = o2satV_ann - o2V_ann 

dm = xr.merge([dm, AOU_ann])


# Monthly amount of moles 

o2V_mon = do.o2 * do.volcello
o2V_mon.name = 'o2V_mon'

o2satV_mon = do.o2sat * do.volcello
o2satV_mon.name = 'o2satV_mon'

AOU_mon = o2satV_mon - o2V_mon 

do = xr.merge([do, AOU_mon])

ValueError: unable to convert unnamed DataArray to a Dataset without providing an explicit name

In [ ]:
# Annual temperature binning

dm = dm.set_coords('thetao')

# Monthly temperature binning 

do = do.set_coords('thetao')